In [1]:
%%writefile TFmapper.py

import sys
import re
from collections import Counter

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

StopWordsFile = 'stop_words_en.txt'
StopWordsList = []

# Makes a list containing the stop words
with open(StopWordsFile, 'r') as f_stop:
    for line in f_stop:
        StopWordsList.append(line.strip())
f_stop.close()


for line in sys.stdin:
    
    total_words_ex_stop, word_list_ex_stop = 0, []
    
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)

    for word in words:
        if word.lower() in StopWordsList:
            continue
        word_list_ex_stop.append(word.lower())

    total_words_ex_stop = len(word_list_ex_stop)
    cnt = Counter(word_list_ex_stop)
    for word in cnt:
        print "%s\t%s\t%d\t%d" % (word, article_id,
                                  cnt[word],total_words_ex_stop)


Overwriting TFmapper.py


In [2]:
%%writefile TFreducer.py

import sys
import re
from collections import Counter
import math

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

# Prints out the word, article_id, and td*idf for each article_id
# containing the word
def output_word_article (word_article_dict):
    num_articles = len(word_article_dict) # number of articles word appears in
    idf = 1.0 / math.log(1 + num_articles) # article frequency statistic

    for word, article_id in word_article_dict:
        word_count, article_word_count = word_article_dict[(word, article_id)]
        tf = (1.0 * word_count) / article_word_count # term frequency
        tf_idf = tf * idf
        print ("%s\t%s\t%f" % (word, article_id, tf_idf))

delim = '\t'
current_word = None
word_article_dict = {}
for line in sys.stdin:
    word, article_id, word_count, article_word_count = (line.strip()).split(delim)    
    if word != current_word:
        if current_word != None:
            output_word_article (word_article_dict)
        current_word = word
        word_article_dict = {}

    word_count, article_word_count = int(word_count), int(article_word_count)
    word_article_dict[(word, article_id)] = (word_count, article_word_count)

Overwriting TFreducer.py


In [4]:
%%bash

OUT_DIR="wordcount_sanj_tfidf_"$(date +"%s%6N")
NUM_REDUCERS=2

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="TF-IDF" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files TFmapper.py,TFreducer.py,/datasets/stop_words_en.txt \
    -mapper "python TFmapper.py" \
    -reducer "python TFreducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

! hdfs dfs -cat ${OUT_DIR}/* | awk '$1=="labor"{print}' | awk '$2==12{print $3}'

0.000350


18/02/26 17:34:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/26 17:34:33 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/26 17:34:33 INFO mapred.FileInputFormat: Total input files to process : 1
18/02/26 17:34:34 INFO mapreduce.JobSubmitter: number of splits:2
18/02/26 17:34:34 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1519657645176_0007
18/02/26 17:34:34 INFO impl.YarnClientImpl: Submitted application application_1519657645176_0007
18/02/26 17:34:34 INFO mapreduce.Job: The url to track the job: http://0e899adab9c8:8088/proxy/application_1519657645176_0007/
18/02/26 17:34:34 INFO mapreduce.Job: Running job: job_1519657645176_0007
18/02/26 17:34:40 INFO mapreduce.Job: Job job_1519657645176_0007 running in uber mode : false
18/02/26 17:34:40 INFO mapreduce.Job:  map 0% reduce 0%
18/02/26 17:34:55 INFO mapreduce.Job:  map 5% reduce 0%
18/02/26 17:35:01 INFO mapreduce.Job:  map 8% reduce 0%
18/02/26 17:35:08 INFO ma